# Segmenting and Clustering Neighborhoods in Toronto - Part III

In [72]:
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import requests

from geopy.geocoders import Nominatim
from pandas.io.json  import json_normalize 
from sklearn.cluster import KMeans

In [4]:
neighs_and_coords = pd.read_csv('list_of_postal_codes_and_coordinates.csv')
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
            len(neighs_and_coords['Borough'].unique()),
            neighs_and_coords.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [10]:
address    = 'Toronto, CA'
geolocator = Nominatim(user_agent="luiz_alberto_capstone_project")
location  = geolocator.geocode(address)
latitude  = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto city are {} {}.'.format(latitude, longitude))

The geographical coordinate of Toronto city are 43.7170226 -79.4197830350134.


In [37]:
map_of_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
neighs_and_coords = neighs_and_coords[neighs_and_coords['Borough'].str.contains('Toronto') == True]
for lat, lng, borough, neighborhood in zip(neighs_and_coords['Latitude'], neighs_and_coords['Longitude'], neighs_and_coords['Borough'], neighs_and_coords['Neighborhood']):
    label = '{} {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186c',
        fill_opacity=0.7
    ).add_to(map_of_toronto)
map_of_toronto    

In [28]:
CLIENT_ID = 'RP1P2BKPRKXDHQZIRAFU50GOPAFWLCQDFTK4NJSKIFVQND0J' # your Foursquare ID
CLIENT_SECRET = 'JYR34RL02SLO3CS25WUNAZE1KJC0BAXLSF5AGSZEJGVVONAL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RP1P2BKPRKXDHQZIRAFU50GOPAFWLCQDFTK4NJSKIFVQND0J
CLIENT_SECRET:JYR34RL02SLO3CS25WUNAZE1KJC0BAXLSF5AGSZEJGVVONAL


In [35]:
neighs_and_coords.loc[37, 'Neighborhood']
neighborhood_latitude = neighs_and_coords.loc[37, 'Latitude'] 
neighborhood_longitude = neighs_and_coords.loc[37, 'Longitude'] 

neighborhood_name = neighs_and_coords.loc[37, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [36]:
# type your answer here
LIMIT  = 100
radius = 500
url    = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url



'https://api.foursquare.com/v2/venues/explore?&client_id=RP1P2BKPRKXDHQZIRAFU50GOPAFWLCQDFTK4NJSKIFVQND0J&client_secret=JYR34RL02SLO3CS25WUNAZE1KJC0BAXLSF5AGSZEJGVVONAL&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [40]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c28dae56a607133f5110ca8'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8daea1f964a520480833e3',
       'name': 'Grover Pub and Grub',
       'location': {'address': '676 Kingston Rd.',
        'crossStreet': 'at Main St.',
        'lat': 43.679181434941015,
        'lng': -79.29721535878515,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.679181434941015,
          'lng': -79.29721535878515}],
    

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [44]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Grover Pub and Grub,Pub,43.679181,-79.297215
1,Starbucks,Coffee Shop,43.678798,-79.298045
2,Williamson Road Playground,Playground,43.674716,-79.297338
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [45]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
neighs_and_coords_venues = getNearbyVenues(names=neighs_and_coords['Neighborhood'],
                                   latitudes=neighs_and_coords['Latitude'],
                                   longitudes=neighs_and_coords['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [49]:
print(neighs_and_coords_venues.shape)
neighs_and_coords_venues.head()

(1691, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Williamson Road Playground,43.674716,-79.297338,Playground
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [69]:
neighs_and_coords_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",19,19,19,19,19,19
Business reply mail Processing Centre969 Eastern,18,18,18,18,18,18
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",13,13,13,13,13,13


In [51]:
print('There are {} uniques categories.'.format(len(neighs_and_coords_venues['Venue Category'].unique())))

There are 231 uniques categories.


In [54]:
neighs_and_coords_onehot = pd.get_dummies(neighs_and_coords_venues[['Venue Category']], prefix="", prefix_sep="")
neighs_and_coords_onehot['Neighborhood'] = neighs_and_coords_venues['Neighborhood'] 
fixed_columns = [neighs_and_coords_onehot.columns[-1]] + list(neighs_and_coords_onehot.columns[:-1])
neighs_and_coords_onehot = neighs_and_coords_onehot[fixed_columns]
neighs_and_coords_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
neighs_and_coords_onehot.shape

(1691, 231)

In [66]:
neighs_and_coords_grouped = neighs_and_coords_onehot.groupby('Neighborhood').mean().reset_index()
neighs_and_coords_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.01
1,Berczy Park,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.052632,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
3,Business reply mail Processing Centre969 Eastern,0.055556,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.0,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.00


In [57]:
neighs_and_coords_grouped.shape

(38, 231)

In [58]:
num_top_venues = 5

for hood in neighs_and_coords_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighs_and_coords_grouped[neighs_and_coords_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.05
3  Italian Restaurant  0.04
4      Farmers Market  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.11
1     Coffee Shop  0.11
2            Café  0.11
3     Yoga Studio  0.05
4             Gym  0.05


----Business reply mail Processing Centre969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2    Recording Studio  0.06
3          Skate Park  0.06
4                 Spa  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.15
1   Airp

In [59]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighs_and_coords_venues_sorted = pd.DataFrame(columns=columns)
neighs_and_coords_venues_sorted['Neighborhood'] = neighs_and_coords_grouped['Neighborhood']

for ind in np.arange(neighs_and_coords_grouped.shape[0]):
    neighs_and_coords_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighs_and_coords_grouped.iloc[ind, :], num_top_venues)

neighs_and_coords_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Gym,Bar,Hotel,Cosmetics Shop,Restaurant
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Pub,Café
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Performing Arts Venue,Pet Store,Climbing Gym,Caribbean Restaurant,Burrito Place
3,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Airport Food Court


In [68]:
kclusters = 5
neighs_and_coords_grouped_clustering = neighs_and_coords_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighs_and_coords_grouped_clustering)
kmeans.labels_[0:10] 

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 2], dtype=int32)

In [70]:
neighs_and_coords_merged = neighs_and_coords
neighs_and_coords_merged['Cluster Labels'] = kmeans.labels_
neighs_and_coords_merged = neighs_and_coords_merged.join(neighs_and_coords_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
neighs_and_coords_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Playground,Coffee Shop,Pub,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Health Food Store,Furniture / Home Store,Pub,Pizza Place,Liquor Store
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,Park,Pet Store,Fish & Chips Shop,Sushi Restaurant,Pub,Ice Cream Shop,Movie Theater,Burrito Place,Steakhouse,Burger Joint
43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Seafood Restaurant,Bookstore,Latin American Restaurant,Cheese Shop,Clothing Store
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Lake,Dim Sum Restaurant,Swim School,Bus Line,Women's Store,Dog Run,Farmers Market,Falafel Restaurant,Event Space


In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(neighs_and_coords_merged['Latitude'], neighs_and_coords_merged['Longitude'], neighs_and_coords_merged['Neighborhood'], neighs_and_coords_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [75]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 0, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,Central Toronto,0,Coffee Shop,Café,Sandwich Place,Pizza Place,Jewish Restaurant,French Restaurant,BBQ Joint,Pub,Indian Restaurant,American Restaurant


In [76]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 1, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,East Toronto,1,Café,Coffee Shop,Bakery,American Restaurant,Italian Restaurant,Seafood Restaurant,Bookstore,Latin American Restaurant,Cheese Shop,Clothing Store
57,Downtown Toronto,1,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bar,Ice Cream Shop,Burger Joint,Thai Restaurant,Chinese Restaurant,Spa
63,Central Toronto,1,Garden,Pool,Women's Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
84,West Toronto,1,Coffee Shop,Café,Pizza Place,Italian Restaurant,Sushi Restaurant,Cosmetics Shop,Diner,Food,Bar,Pub


In [77]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 2, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,2,Playground,Coffee Shop,Pub,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
41,East Toronto,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Health Food Store,Furniture / Home Store,Pub,Pizza Place,Liquor Store
42,East Toronto,2,Park,Pet Store,Fish & Chips Shop,Sushi Restaurant,Pub,Ice Cream Shop,Movie Theater,Burrito Place,Steakhouse,Burger Joint
44,Central Toronto,2,Park,Lake,Dim Sum Restaurant,Swim School,Bus Line,Women's Store,Dog Run,Farmers Market,Falafel Restaurant,Event Space
45,Central Toronto,2,Gym / Fitness Center,Hotel,Sandwich Place,Burger Joint,Food & Drink Shop,Dance Studio,Asian Restaurant,Breakfast Spot,Park,Grocery Store
46,Central Toronto,2,Clothing Store,Coffee Shop,Sporting Goods Shop,Yoga Studio,Park,Sandwich Place,Salon / Barbershop,Rental Car Location,Chinese Restaurant,Cosmetics Shop
47,Central Toronto,2,Dessert Shop,Sandwich Place,Pizza Place,Coffee Shop,Café,Sushi Restaurant,Seafood Restaurant,Italian Restaurant,Diner,Farmers Market
48,Central Toronto,2,Restaurant,Playground,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
49,Central Toronto,2,Coffee Shop,Pub,American Restaurant,Sushi Restaurant,Fried Chicken Joint,Bagel Shop,Sports Bar,Supermarket,Pizza Place,Light Rail Station
50,Downtown Toronto,2,Park,Playground,Trail,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [78]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 3, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Downtown Toronto,3,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Boutique,Airport Gate,Airport,Airport Food Court


In [79]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 4, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Downtown Toronto,4,Coffee Shop,Restaurant,Seafood Restaurant,Café,Cocktail Bar,Pub,Hotel,Italian Restaurant,Beer Bar,Cosmetics Shop
